# Proceso ETL - Carga de Datos a PostgreSQL

Este notebook documenta el proceso completo de carga de datos desde archivos CSV a PostgreSQL, incluyendo el flujo de ejecución y los resultados obtenidos.


## 📋 Resumen del Proceso

El proceso ETL (Extract, Transform, Load) se ejecuta en dos fases principales:

1. **Fase 1: Creación del Esquema** - Crea todas las tablas en PostgreSQL usando SQLAlchemy ORM
2. **Fase 2: Carga de Datos** - Lee archivos CSV, transforma los datos y los carga en las tablas creadas


## 🔄 Flujo de Ejecución del Código

### Punto de Entrada: `main.py`

El proceso comienza en `Avance1/main.py`, que actúa como orquestador principal:

```python
# 1. Configuración inicial de paths
path_manager = PathManager.get_instance()  # Singleton
path_manager.setup_sys_path()  # Configura sys.path

# 2. Import de funciones
from Models.create_tables import create_all_tables
from ETL.load_data import load_data

# 3. Ejecución del proceso
def main():
    # Paso 1: Crear esquema
    create_all_tables()
    
    # Paso 2: Cargar datos
    for config in tables_config:
        load_data(config['file'], config['table'])
```


### Paso 1: Creación del Esquema (`Models/create_tables.py`)

**Flujo de ejecución:**

1. **Obtener conexión a la base de datos:**
   ```python
   db = DBConnector.get_instance()  # Singleton
   engine = db.get_engine()
   ```

2. **Crear todas las tablas:**
   ```python
   Base.metadata.create_all(engine)
   ```
   - Utiliza los modelos ORM definidos en `Models/models.py`
   - Crea 11 tablas con sus constraints, foreign keys, enums y validaciones
   - Aplica CheckConstraints para validaciones numéricas
   - Crea tipos ENUM nativos de PostgreSQL para campos de estado


### Paso 2: Carga de Datos (`ETL/load_data.py`)

**Flujo de ejecución para cada archivo CSV:**

1. **Obtener ruta del archivo CSV:**
   ```python
   csv_path = path_manager.get_csv_path(file_name)
   # Ejemplo: 'DataSet/CSV/2.Usuarios.csv'
   ```

2. **Leer archivo CSV con pandas:**
   ```python
   df = pd.read_csv(csv_path, encoding=ETLConfig.CSV_ENCODING)  # 'utf-8'
   ```

3. **Estandarizar nombres de columnas:**
   ```python
   df.columns = [clean_column_name(col) for col in df.columns]
   # Convierte camelCase a snake_case: 'OrdenID' -> 'orden_id'
   ```

4. **Obtener conexión a la base de datos:**
   ```python
   db = DBConnector.get_instance()  # Singleton
   ```

5. **Insertar datos en PostgreSQL usando COPY:**
   ```python
   # Obtener conexión raw de psycopg2 usando el context manager del DBConnector
   with db.get_raw_connection() as conn:
       cursor = conn.cursor()
       
       # Convertir DataFrame a CSV en memoria (StringIO)
       csv_buffer = io.StringIO()
       df.to_csv(
           csv_buffer,
           index=False,  # No incluir el índice
           header=False,  # COPY no necesita header
           sep=',',  # Separador de columnas
           na_rep='',  # Representación de valores nulos
           quoting=1,  # QUOTE_MINIMAL
           escapechar='\\',  # Carácter de escape
           doublequote=True,  # Escapar comillas dobles
           lineterminator='\n'  # Terminador de línea Unix
       )
       csv_buffer.seek(0)
       
       # Obtener nombres de columnas
       columns = ', '.join(df.columns)
       
       # Ejecutar COPY FROM usando psycopg2
       cursor.copy_expert(
           f"COPY {table_name} ({columns}) FROM STDIN WITH (FORMAT CSV, DELIMITER ',', NULL '', QUOTE '\"', ESCAPE '\\')",
           csv_buffer
       )
       
       # Confirmar la transacción
       conn.commit()
       cursor.close()
   ```
   - Usa el comando COPY nativo de PostgreSQL (máxima eficiencia)
   - Procesa todos los datos de una vez (más rápido que INSERT individuales)
   - Carga directa desde memoria sin archivos temporales
   - Transaccional (todo o nada)
   - Manejo automático de errores con rollback


## 📊 Resultado de la Ejecución

### Salida de la Consola

```
INICIANDO PROCESO ETL COMPLETO

PASO 1: Creando esquema de base de datos...
CREANDO ESQUEMA DE BASE DE DATOS CON SQLALCHEMY ORM
Todas las tablas creadas exitosamente:
Esquema de base de datos creado exitosamente

PASO 2: Cargando datos desde archivos CSV...

Leyendo archivo: 2.Usuarios.csv
   ✓ Archivo leído. Filas: 1000, Columnas: 5
   ✓ Nombres de columnas estandarizados
   Cargando datos a la tabla 'usuarios' usando COPY...
   ✓ Datos cargados exitosamente a 'usuarios' (1000 filas)

Leyendo archivo: 3.Categorias.csv
   ✓ Archivo leído. Filas: 12, Columnas: 2
   ✓ Nombres de columnas estandarizados
   Cargando datos a la tabla 'categorias' usando COPY...
   ✓ Datos cargados exitosamente a 'categorias' (12 filas)

Leyendo archivo: 4.Productos.csv
   ✓ Archivo leído. Filas: 36, Columnas: 5
   ✓ Nombres de columnas estandarizados
   Cargando datos a la tabla 'productos' usando COPY...
   ✓ Datos cargados exitosamente a 'productos' (36 filas)

Leyendo archivo: 5.ordenes.csv
   ✓ Archivo leído. Filas: 10000, Columnas: 4
   ✓ Nombres de columnas estandarizados
   Cargando datos a la tabla 'ordenes' usando COPY...
   ✓ Datos cargados exitosamente a 'ordenes' (10000 filas)

Leyendo archivo: 6.detalle_ordenes.csv
   ✓ Archivo leído. Filas: 10000, Columnas: 4
   ✓ Nombres de columnas estandarizados
   Cargando datos a la tabla 'detalle_ordenes' usando COPY...
   ✓ Datos cargados exitosamente a 'detalle_ordenes' (10000 filas)

Leyendo archivo: 7.direcciones_envio.csv
   ✓ Archivo leído. Filas: 1000, Columnas: 9
   ✓ Nombres de columnas estandarizados
   Cargando datos a la tabla 'direcciones_envio' usando COPY...
   ✓ Datos cargados exitosamente a 'direcciones_envio' (1000 filas)

Leyendo archivo: 8.carrito.csv
   ✓ Archivo leído. Filas: 5000, Columnas: 4
   ✓ Nombres de columnas estandarizados
   Cargando datos a la tabla 'carrito' usando COPY...
   ✓ Datos cargados exitosamente a 'carrito' (5000 filas)

Leyendo archivo: 9.metodos_pago.csv
   ✓ Archivo leído. Filas: 7, Columnas: 2
   ✓ Nombres de columnas estandarizados
   Cargando datos a la tabla 'metodos_pago' usando COPY...
   ✓ Datos cargados exitosamente a 'metodos_pago' (7 filas)

Leyendo archivo: 10.ordenes_metodospago.csv
   ✓ Archivo leído. Filas: 10000, Columnas: 3
   ✓ Nombres de columnas estandarizados
   Cargando datos a la tabla 'ordenes_metodos_pago' usando COPY...
   ✓ Datos cargados exitosamente a 'ordenes_metodos_pago' (10000 filas)

Leyendo archivo: 11.resenas_productos.csv
   ✓ Archivo leído. Filas: 7172, Columnas: 5
   ✓ Nombres de columnas estandarizados
   Cargando datos a la tabla 'resenas_productos' usando COPY...
   ✓ Datos cargados exitosamente a 'resenas_productos' (7172 filas)

Leyendo archivo: 12.historial_pagos.csv
   ✓ Archivo leído. Filas: 10000, Columnas: 5
   ✓ Nombres de columnas estandarizados
   Cargando datos a la tabla 'historial_pagos' usando COPY...
   ✓ Datos cargados exitosamente a 'historial_pagos' (10000 filas)

PROCESO ETL COMPLETADO EXITOSAMENTE

Resumen:
   - Tablas creadas: 11
   - Archivos CSV procesados: 11
```


## 📈 Estadísticas del Proceso

| Archivo CSV | Tabla | Filas Cargadas | Columnas |
|-------------|-------|----------------|----------|
| `2.Usuarios.csv` | `usuarios` | 1,000 | 5 |
| `3.Categorias.csv` | `categorias` | 12 | 2 |
| `4.Productos.csv` | `productos` | 36 | 5 |
| `5.ordenes.csv` | `ordenes` | 10,000 | 4 |
| `6.detalle_ordenes.csv` | `detalle_ordenes` | 10,000 | 4 |
| `7.direcciones_envio.csv` | `direcciones_envio` | 1,000 | 9 |
| `8.carrito.csv` | `carrito` | 5,000 | 4 |
| `9.metodos_pago.csv` | `metodos_pago` | 7 | 2 |
| `10.ordenes_metodospago.csv` | `ordenes_metodos_pago` | 10,000 | 3 |
| `11.resenas_productos.csv` | `resenas_productos` | 7,172 | 5 |
| `12.historial_pagos.csv` | `historial_pagos` | 10,000 | 5 |

**Total:** 55,227 registros cargados en 11 tablas


## 🔧 Componentes Técnicos Utilizados

### 1. Patrón Singleton
- **`DBConnector`**: Gestión única de conexión a PostgreSQL
  - Método `get_raw_connection()`: Context manager para conexiones raw de psycopg2 (usado para COPY)
- **`PathManager`**: Gestión centralizada de rutas del proyecto

### 2. Configuración Centralizada
- **`ETLConfig`**: Todos los parámetros configurables en un solo lugar
  - `CSV_ENCODING = 'utf-8'` - Encoding de archivos CSV
  - **Nota**: COPY de PostgreSQL no requiere parámetros de chunksize o método de inserción (es más eficiente que INSERT individuales)

### 3. Transformaciones Aplicadas
- **Estandarización de nombres**: `clean_column_name()`
  - Convierte camelCase a snake_case
  - Ejemplo: `OrdenID` → `orden_id`
  - Convierte a minúsculas
  - Elimina caracteres especiales

### 3. Carga de Datos con COPY
- **Comando COPY nativo de PostgreSQL**: Implementado vía `psycopg2.copy_expert()`
  - Máxima eficiencia: COPY es el método más rápido para cargar datos
  - Procesamiento directo: Carga desde memoria (StringIO) sin archivos temporales
  - Transaccional: Manejo automático de commit/rollback
  - Context manager: Uso de `db.get_raw_connection()` para manejo seguro de conexiones

### 4. Validaciones Implementadas
- **CheckConstraints**: Validaciones numéricas (>= 0)
- **Enums nativos**: Validación de valores de estado
- **Foreign Keys**: Integridad referencial


## 📁 Estructura de Archivos Ejecutados

```
Avance1/
├── main.py                    # Orquestador principal
│
├── Models/
│   ├── create_tables.py      # Paso 1: Creación de esquema
│   ├── models.py              # Modelos ORM (11 tablas)
│   └── enums.py               # Enumeraciones (EstadoOrden, EstadoPago)
│
├── ETL/
│   └── load_data.py           # Paso 2: Carga de datos
│
└── Utils/
    ├── path_manager.py       # Gestión de rutas (Singleton)
    ├── config.py              # Configuración centralizada
    └── clean_column_name.py  # Transformación de nombres
```


## ✅ Validaciones y Constraints Aplicados

### CheckConstraints (Validaciones Numéricas)
- `precio >= 0` (productos)
- `stock >= 0` (productos)
- `total >= 0` (ordenes)
- `cantidad >= 0` (detalle_ordenes, carrito)
- `precio_unitario >= 0` (detalle_ordenes)
- `monto_pagado >= 0` (ordenes_metodos_pago)
- `monto >= 0` (historial_pagos)
- `calificacion >= 1 AND calificacion <= 5` (resenas_productos)

### Enums Nativos de PostgreSQL
- **`estado_orden`**: Pendiente, Enviado, Completado, Cancelado
- **`estado_pago`**: Procesando, Pagado, Fallido, Reembolsado


## 🎯 Resultado Final

✅ **11 tablas creadas exitosamente**

✅ **11 archivos CSV procesados exitosamente**

✅ **55,227 registros cargados en PostgreSQL**

✅ **Todas las validaciones y constraints aplicados correctamente**

✅ **Integridad referencial mantenida mediante Foreign Keys**

✅ **Tipos de datos ajustados para preservar integridad semántica**
